# 1. Carga de librerias 

In [ ]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 2. rutas.

In [ ]:
# Ruta archivos entrada
R_Ms_ADRES_EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0019052025.TXT"
R_Ms_ADRES_EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-2\EPS025MS0019052025.TXT"
R_S3 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\S3\2025-2\S3EPS02513052025.TXT"

# Ruta archivos salida
Carpeta = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\05_Mayo\20"

In [ ]:
Fecha = "20/05/2025"

# 3. Carga Dataframes

In [ ]:
new_columns = ["AFL_ID", "ENT_ID", "TPS_IDN_ID_CF", "HST_IDN_NUMERO_IDENTIFICACION_CF", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO", "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "AFL_PAIS_NACIMIENTO", "AFL_MUNICIPIO_NACIMIENTO", "AFL_NACIONALIDAD", "AFL_SEXO_IDENTIFICACION", "AFL_DISCAPACIDAD", "TPS_AFL_ID", "TPS_PRN_ID", "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "NUMEROFICHASISBEN", "TPS_CND_BNF_ID", "DPR_ID", "MNC_ID", "ZNS_ID", "AFL_FECHA_AFILIACION_SGSSS", "AFC_FECHA_INICIO", "NUMERO CONTRATO", "FECHADE INICIO DEL CONTRATO", "CNT_AFL_TPS_GRP_PBL_ID", "CNT_AFL_TPS_PRT_ETN_ID", "TPS_MDL_SBS_ID", "TPS_EST_AFL_ID", "CND_AFL_FECHA_INICIO", "CND_AFL_FECHA_INICIO", "GRP_FML_COTIZANTE_ID", "PORTABILIDAD", "COD_IPS_P", "MTDLG_G_P", "SUB_SISBEN_IV", "MARCASISBENIV+MARCASISBENIII", "CRUCE_BDEX_RNEC"]

Df_EPS025 = pd.read_csv(R_Ms_ADRES_EPS025, sep=',', header=None, dtype=str, encoding='ANSI')
Df_EPS025.columns = new_columns

Df_EPSC25 = pd.read_csv(R_Ms_ADRES_EPSC25, sep=',', header=None, dtype=str, encoding='ANSI')
Df_EPSC25.columns = new_columns

In [ ]:
new_columns = ["ENT_ID", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO", "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "TPS_IDN_ID_2", "HST_IDN_NUMERO_IDENTIFICACION_2", "AFL_PRIMER_APELLIDO_2", "AFL_SEGUNDO_APELLIDO_2", "AFL_PRIMER_NOMBRE_2", "AFL_SEGUNDO_NOMBRE_2", "AFL_FECHA_NACIMIENTO_2", "TPS_GNR_ID_2", "DPR_ID", "MNC_ID", "ZNS_ID", "FECHA_AFILIACION_MOVILIDAD", "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "TIPO_TRASLADO", "CND_AFL_SBS_METODOLOGIA", "CND_AFL_SBS_SUBGRUPO_SIV", "CON_DISCAPACIDAD", "TPS_IDN_CF_ID", "HST_IDN_NUMERO_CF_IDENTIFICACION", "TPS_PRN_ID", "TPS_AFL_ID", "TPS_MDL_SBS_ID", "ENT_ID_ORIGEN", "TPS_ETN_ID", "NOM_RESGUARDO_INDIGENA", "PAIS_NACIMIENTO", "LUGAR_NACIMIENTO", "NACIONALIDAD", "SEXO_IDENTIFICACION", "TIPO_DISCAPACIDAD", "GlOSA"]
Df_S3 = pd.read_csv(R_S3, sep=',', header=None, dtype=str, encoding='ANSI')
Df_S3.columns = new_columns
print("Número de registros en Df_S3:", Df_S3.shape[0])

# Agregar columna "Enviar" con un valor inicial vacío
Df_S3['Enviar'] = ''


# 4. Limpier datos

In [ ]:
# 1. Concatenar uno debajo del otro
DF_ADRES = pd.concat(
    [Df_EPS025, Df_EPSC25],
    ignore_index=True,   # reindexa de 0…n-1
    sort=False           # evita warnings si el orden de columnas coincide
)

# 2. (Opcional) borrar los DataFrames originales para liberar memoria
del Df_EPS025, Df_EPSC25

# Seleccionar las columnas de DF_ADRES a transferir, junto con las columnas clave
cols_transfer = ["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "ENT_ID", "TPS_EST_AFL_ID"]
df_transfer = DF_ADRES[cols_transfer].drop_duplicates()

# Hacemos un merge de Df_S3 con df_transfer mediante las columnas clave
Df_S3 = Df_S3.merge(
    df_transfer,
    on=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"],
    how="left",
    suffixes=("", "_from_adres")
)

# Si se desea actualizar el valor de ENT_ID en Df_S3 usando el de DF_ADRES,
# se puede hacer lo siguiente. De lo contrario, se conservará el valor original:
Df_S3["ENT_ID"] = Df_S3["ENT_ID_from_adres"].combine_first(Df_S3["ENT_ID"])

# Renombrar la columna importada TPS_EST_AFL_ID (si se quiere conservar con ese nombre)
Df_S3.rename(columns={"TPS_EST_AFL_ID": "TPS_EST_AFL_ID_from_adres"}, inplace=True)

# Opcional: se puede eliminar la columna auxiliar ENT_ID_from_adres ya que su valor fue transferido
Df_S3.drop(columns=["ENT_ID_from_adres"], inplace=True)

In [ ]:
print("Número de registros en Df_S3:", Df_S3.shape[0])

# 1. Definimos la máscara con la lógica (cond1 AND cond2)  OR  cond3
mask = (
    (Df_S3["ENT_ID"] == "EPS025") &
    (Df_S3["TPS_EST_AFL_ID_from_adres"] == "AC")
) | Df_S3["TIPO_TRASLADO"].isin(["0", "1", "2"])

# 2. Extraemos los registros a enviar
DF_No_Enviar = Df_S3.loc[mask].copy()

# 3. Eliminamos esos mismos registros del DataFrame original
Df_S3 = Df_S3.loc[~mask].copy()

print("Número de registros en DF_No_Enivar:", DF_No_Enviar.shape[0])
print("Número de registros en Df_S3:", Df_S3.shape[0])

# 5. Validar Datos

In [ ]:
# Agregar la columna "No_Glosas" contando las glosas separadas por ";"
Df_S3['No_Glosas'] = Df_S3['GlOSA'].apply(
    lambda x: len(x.rstrip(';').split(';')) if isinstance(x, str) else 0
)

# Imprimir los valores únicos de la nueva columna
print(Df_S3['No_Glosas'].unique())

In [ ]:
# Extraer los primeros 6 caracteres de la columna 'GlOSA' y guardarlos en 'Glosa_Actual'
Df_S3['Glosa_Actual'] = Df_S3['GlOSA'].str[:6]

# Duplicar la columna 'GlOSA' en una nueva columna 'GlOSA_2'
Df_S3['GlOSA_2'] = Df_S3['GlOSA']

# Imprimir los valores únicos y la cantidad de registros de cada uno en la columna "Glosa_Actual"
print("Valores únicos en Glosa_Actual:")
print(Df_S3['Glosa_Actual'].unique())

print("\nCantidad de registros por cada valor en Glosa_Actual:")
print(Df_S3['Glosa_Actual'].value_counts())

# 5.1. Glosa GN0368

In [ ]:

def limpiar_glosa_GN0368(row):
    """
    Limpia la glosa GN0368 y ajusta la fecha en formato dd/mm/YYYY.
    """
    glosa_str = row.get("GlOSA_2", "")
    if not glosa_str:
        return row

    # Separar tokens por ";" y filtrar vacíos
    tokens = [t for t in glosa_str.split(";") if t]

    # Buscar token GN0368(
    token_gn = next((t for t in tokens if t.startswith("GN0368(")), None)
    if not token_gn:
        return row

    # Extraer fecha en formato dd/mm/YYYY dentro de GN0368(...)
    m = re.search(r"GN0368\([^()]*?(\d{2}/\d{2}/\d{4})\)", token_gn)
    if not m:
        return row

    fecha_str = m.group(1)
    fecha_dt = datetime.strptime(fecha_str, "%d/%m/%Y")
    # Sumar un día y formatear de nuevo como dd/mm/YYYY
    nueva_fecha = (fecha_dt + timedelta(days=1)).strftime("%d/%m/%Y")

    row["FECHA_AFILIACION_MOVILIDAD"] = nueva_fecha

    # Eliminar el token GN0368 de la lista
    restantes = [t for t in tokens if not t.startswith("GN0368(")]
    # Reconstruir GlOSA_2
    row["GlOSA_2"] = ";".join(restantes) + (";" if restantes else "")

    # Actualizar Glosa_Actual
    if row.get("No_Glosas", 0) > 1 and restantes:
        row["Glosa_Actual"] = restantes[0].split("(")[0]
    else:
        row["Glosa_Actual"] = ""

    return row

# Aplicación sobre el DataFrame
Df_S3 = Df_S3.apply(limpiar_glosa_GN0368, axis=1)


# 5.2. Glosa GN0369

In [ ]:
def limpiar_glosa_369(row):
    glosa_str = row.get("GlOSA_2", "")
    if not glosa_str or "GN0369(" not in glosa_str:
        return row

    # split y quitamos vacíos
    tokens = [tok for tok in glosa_str.split(";") if tok.strip()]
    # buscamos la glosa GN0369
    token = next((tok for tok in tokens if tok.startswith("GN0369(")), None)
    if not token:
        return row

    # regex para extraer meses y fecha final
    m = re.search(r"GN0369\([^|]*\|(\d+)\|[^|]*\|(\d{2}/\d{2}/\d{4})\)", token)
    if not m:
        return row

    meses = int(m.group(1))
    fecha_base = datetime.strptime(m.group(2), "%d/%m/%Y")

    # sumamos meses y un día
    nueva_fecha_dt = fecha_base + relativedelta(months=meses) + timedelta(days=1)
    nueva_fecha = nueva_fecha_dt.strftime("%d/%m/%Y")

    # actualizamos la columna de movilidad
    row["FECHA_AFILIACION_MOVILIDAD"] = nueva_fecha

    # eliminamos la glosa procesada
    tokens = [tok for tok in tokens if not tok.startswith("GN0369(")]
    row["GlOSA_2"] = ( ";".join(tokens) + ";" ) if tokens else ""

    # ajustamos Glosa_Actual
    if row.get("No_Glosas", 0) > 1:
        # si quedan otras glosas, tomamos la primera antes del "("
        primera = tokens[0]
        row["Glosa_Actual"] = primera.split("(")[0]
    else:
        row["Glosa_Actual"] = ""

    return row

# Para aplicarlo al DataFrame:
Df_S3 = Df_S3.apply(limpiar_glosa_369, axis=1)


# 5.3. Glosa GN0421

In [ ]:
def limpiar_glosa_421(row):
    glosa_str = row.get("GlOSA_2", "")
    if not glosa_str:
        return row

    # Separamos tokens y buscamos GN0421(
    tokens = [tok for tok in glosa_str.split(";") if tok]
    token_421 = next((tok for tok in tokens if tok.startswith("GN0421(")), None)
    if not token_421:
        return row

    # Extraemos la fecha
    m = re.search(r"GN0421\((\d{2}/\d{2}/\d{4})\)", token_421)
    if not m:
        return row
    fecha_base = datetime.strptime(m.group(1), "%d/%m/%Y")

    # Sumamos un día
    nueva_fecha = (fecha_base + timedelta(days=1)).strftime("%d/%m/%Y")
    row["FECHA_AFILIACION_MOVILIDAD"] = nueva_fecha

    # Eliminamos el token procesado
    tokens = [tok for tok in tokens if not tok.startswith("GN0421(")]
    row["GlOSA_2"] = (";".join(tokens) + ";") if tokens else ""

    # Ajustamos Glosa_Actual
    if row.get("No_Glosas", 0) > 1 and tokens:
        row["Glosa_Actual"] = tokens[0].split("(")[0]
    else:
        row["Glosa_Actual"] = ""

    return row

# Para aplicarlo:
Df_S3 = Df_S3.apply(limpiar_glosa_421, axis=1)


# 5.4. Glosa GN0084

In [ ]:
def limpiar_glosa_0084(row):
    glosa_str = row.get("GlOSA_2", "")
    if not glosa_str:
        return row

    # Partimos tokens y buscamos GN0084(
    tokens = [tok for tok in glosa_str.split(";") if tok]
    token_84 = next((tok for tok in tokens if tok.startswith("GN0084(")), None)
    if not token_84:
        return row

    # Extraemos la fecha tras el '|'
    m = re.search(r"GN0084\([^|]+\|(\d{2}/\d{2}/\d{4})\)", token_84)
    if m:
        fecha_glosa = datetime.strptime(m.group(1), "%d/%m/%Y")
        # Intentamos parsear la fecha existente en FECHA_AFILIACION_MOVILIDAD
        try:
            fecha_actual = datetime.strptime(row.get("FECHA_AFILIACION_MOVILIDAD",""), "%d/%m/%Y")
        except Exception:
            fecha_actual = None

        # Solo si la fecha actual es anterior a la de la glosa, la actualizamos
        if fecha_actual and fecha_actual < fecha_glosa:
            nueva = (fecha_glosa + timedelta(days=1)).strftime("%d/%m/%Y")
            row["FECHA_AFILIACION_MOVILIDAD"] = nueva

    # Eliminamos siempre el token GN0084
    tokens = [tok for tok in tokens if not tok.startswith("GN0084(")]
    row["GlOSA_2"] = (";".join(tokens) + ";") if tokens else ""

    # Ajustamos Glosa_Actual
    if row.get("No_Glosas", 0) > 1 and tokens:
        row["Glosa_Actual"] = tokens[0].split("(")[0]
    else:
        row["Glosa_Actual"] = ""

    return row

# Para aplicarlo sobre tu DataFrame:
Df_S3 = Df_S3.apply(limpiar_glosa_0084, axis=1)


# 6. Guardar informacion

In [ ]:
output_file = Path(Carpeta) / "resultados.xlsx"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    Df_S3.to_excel(writer, sheet_name="Df_S3", index=False)
    DF_No_Enviar.to_excel(writer, sheet_name="DF_No_Enviar", index=False)
    DF_ADRES.to_excel(writer, sheet_name="DF_ADRES", index=False)

print("Archivo Excel guardado en:", output_file)

In [ ]:
print(Df_S3['Glosa_Actual'].value_counts())